### Preppin' Data Challenge: 2023 Week6 - DSB Customer Ratings

<br>
Input the data

In [519]:
import pandas as pd
import numpy as np
import re  #importing regex

df = pd.read_csv('PD2023_WK06_DSB_CUSTOMER_SURVEY.csv')

<br>
Reshape the data so we have 5 rows for each customer, with responses for the Mobile App and Online Interface being in separate fields on the same row.

- Clean the question categories so they don't have the platform in from of them e.g. Mobile App - Ease of Use should be simply Ease of Use

In [520]:
#unpivoting the dataframe
df = pd.melt(df, id_vars='CUSTOMER_ID', var_name='Response', value_name='Rating')

In [521]:
#Extracting Interface Value from Response column using regex
df['Interface'] = df['Response'].str.extract(r'(.+)___')

#Extracting Reponse Value from Response column using regex
df['Response'] = df['Response'].str.extract(r'.+___(.+)')

In [522]:
#Pivoting 'Interface' row value to column
#reset_index() is used to  push the grouped-on values of a columns back into each records in the result

df = df.pivot_table(index=['CUSTOMER_ID', 'Response'], columns = 'Interface', values = 'Rating').reset_index()

<br>
Exclude the Overall Ratings, these were incorrectly calculated by the system

In [523]:
df = df.loc[df['Response'] != 'OVERALL_RATING']

<br>
Calculate the Average Ratings for each platform for each customer 

In [524]:
df = df.groupby(['CUSTOMER_ID'], as_index=False).agg({'MOBILE_APP':'mean', 'ONLINE_INTERFACE':'mean'})

<br>
Calculate the difference in Average Rating between Mobile App and Online Interface for each customer

In [525]:
df['Difference Average Rating'] = df['MOBILE_APP'] - df['ONLINE_INTERFACE']

<br>
Catergorise customers as being:

- Mobile App Superfans if the difference is greater than or equal to 2 in the Mobile App's favour
- Mobile App Fans if difference >= 1
- Online Interface Fan
- Online Interface Superfan
- Neutral if difference is between 0 and 1

In [526]:
df['Preference'] = df['Difference Average Rating'].apply(
                                                        lambda x: 'Mobile Superfans' if x >= 2 
                                                        else 'Mobile Fans' if x >= 1 and x < 2 
                                                        else 'Online Fans' if x <=-1 and x>-2
                                                        else 'Online Superfan' if x <= -2
                                                        else 'Neutral')


<br>
Calculate the Percent of Total customers in each category, rounded to 1 decimal place

In [527]:
# Numbers of customers by each preference
df = df.groupby('Preference', as_index=False).count()

# Total of all customers in each record
df['Total'] = df['MOBILE_APP'].sum()

#Calculating % of total customers in each category
df['% of Total'] = df['MOBILE_APP']/df['Total']*100

#Round to 1 decimal
df['% of Total'] = df['% of Total'].round(1)

In [528]:
column_order = ['Preference', '% of Total']
df = df[column_order]
df = df.reset_index()

<br>
Output the data

In [529]:
df.to_csv("PD2023_W6_Output.csv", index=False)

In [530]:
#Verifying the output
Output = pd.read_csv("PD2023_W6_Output.csv")
Output

,index,Preference,% of Total
0,0,Mobile Fans,16.4
1,1,Mobile Superfans,2.6
2,2,Neutral,63.7
3,3,Online Fans,14.7
4,4,Online Superfan,2.6
